In [1]:
import numpy as np
import xarray as xr
import geopandas as gpd
import xagg as xa
import salem

import python_utils as pu

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [2]:
############
### Dask ###
############
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers = 10)
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 10
Total threads: 10,Total memory: 187.41 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36747,Workers: 10
Dashboard: /proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 187.41 GiB
Comm: tcp://127.0.0.1:44197,Total threads: 1
Dashboard: /proxy/43195/status,Memory: 18.74 GiB
Nanny: tcp://127.0.0.1:36043,


## Solar

In [3]:
# Read 2006 only
ds = pu._preprocess_tgw(xr.open_dataset(f"{pu.tgw_path}/historical_1980_2019/hourly/tgw_wrf_historical_hourly_1980-01-01_01_00_00.nc"))

In [11]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (Time: 168, south_north: 299, west_east: 424)
Coordinates:
    XLAT       (Time, south_north, west_east) float32 85MB ...
    XLONG      (Time, south_north, west_east) float32 85MB ...
  * Time       (Time) datetime64[ns] 1kB 1980-01-01T01:00:00 ... 1980-01-08
Dimensions without coordinates: south_north, west_east
Data variables: (12/26)
    Times      (Time) |S19 3kB ...
    LU_INDEX   (Time, south_north, west_east) float32 85MB ...
    Q2         (Time, south_north, west_east) float32 85MB ...
    T2         (Time, south_north, west_east) float32 85MB ...
    PSFC       (Time, south_north, west_east) float32 85MB ...
    U10        (Time, south_north, west_east) float32 85MB ...
    ...         ...
    HAILNC     (Time, south_north, west_east) float32 85MB ...
    SWDOWN     (Time, south_north, west_east) float32 85MB ...
    GLW        (Time, south_north, west_east) float32 85MB ...
    UST        (Time, south_north, west_east) float32 85MB ...
    SNOWC      (Time, south_north, west_east) float32 85MB ...
    SR         (Time, south_north, west_east) float32 85MB ...
Attributes: (12/142)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      1980-01-01_00:00:00
    SIMULATION_START_DATE:           1979-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        425
    SOUTH-NORTH_GRID_DIMENSION:      300
    BOTTOM-TOP_GRID_DIMENSION:       33
    ...                              ...
    ISLAKE:                          -1
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

In [9]:
# NYISO shapefile
gdf = gpd.read_file('../../data/nyiso/shapefiles/NYISO_Load_Zone_Dissolved.shp')

In [10]:
# Get overlap between pixels and polygons
weightmap = xa.pixel_overlaps(ds,gdf)

creating polygons for each pixel...


NameError: No valid lat/lon variables found in the dataset.

In [ ]:

# Aggregate data in [ds] onto polygons
aggregated = xa.aggregate(ds,weightmap)

In [4]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (Time: 168, south_north: 299, west_east: 424)
Coordinates:
    XLAT       (Time, south_north, west_east) float32 85MB ...
    XLONG      (Time, south_north, west_east) float32 85MB ...
  * Time       (Time) datetime64[ns] 1kB 1980-01-01T01:00:00 ... 1980-01-08
Dimensions without coordinates: south_north, west_east
Data variables: (12/26)
    Times      (Time) |S19 3kB ...
    LU_INDEX   (Time, south_north, west_east) float32 85MB ...
    Q2         (Time, south_north, west_east) float32 85MB ...
    T2         (Time, south_north, west_east) float32 85MB ...
    PSFC       (Time, south_north, west_east) float32 85MB ...
    U10        (Time, south_north, west_east) float32 85MB ...
    ...         ...
    HAILNC     (Time, south_north, west_east) float32 85MB ...
    SWDOWN     (Time, south_north, west_east) float32 85MB ...
    GLW        (Time, south_north, west_east) float32 85MB ...
    UST        (Time, south_north, west_east) float32 85MB ...
    SNOWC      (Time, south_north, west_east) float32 85MB ...
    SR         (Time, south_north, west_east) float32 85MB ...
Attributes: (12/142)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      1980-01-01_00:00:00
    SIMULATION_START_DATE:           1979-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        425
    SOUTH-NORTH_GRID_DIMENSION:      300
    BOTTOM-TOP_GRID_DIMENSION:       33
    ...                              ...
    ISLAKE:                          -1
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

In [ ]:
# Load desired variable
ds = ds[['GLW']].load()

In [ ]:
# Compute the absolute difference
target_lat = 41.55
target_lon = -74.15

distance = np.sqrt((ds.XLAT - target_lat) ** 2 + (ds.XLONG - target_lon) ** 2)

# Find the index of the minimum distance
min_idx = distance.argmin(dim=["south_north", "west_east"])

# Select the corresponding data
ds_selected = ds.isel(south_north=min_idx['south_north'], west_east=min_idx['west_east'])

ds_selected['GLW'].plot()